#### get paperID from database of top5000

connect database

In [1]:
field = 'visualization'
import mysql.connector
conn = mysql.connector.connect(host="192.168.0.140",
                                user="root",
                                password="root",
                                database=f"scigene_{field}_field")
cursor = conn.cursor()

connMAG = mysql.connector.connect(host="192.168.0.140",
                                user="root",
                                password="root",
                                database=f"MACG")
cursorMAG = connMAG.cursor()

fetch paperID

In [2]:
cursor.execute('select paperID, title from papers_field')
ret = cursor.fetchall()
paperIDList = [r[0] for r in ret]

import pandas as pd
df = pd.DataFrame(ret, columns=['paperID', 'title'])

In [3]:
paperIDPath = '/home/sy/arc/create_field/out/scigene_CG_field/papers.txt'
with open(paperIDPath, 'r') as f:
    paperIDList = [s.strip() for s in f.readlines()]


In [4]:
abInfoList = []
ttInfoList = []

batchCount = 10000
batchSize = len(paperIDList) // batchCount
for i in range(batchCount):
    paperIDSetStr = ','.join([f'"{papID}"' for papID in paperIDList[i * batchSize : (i + 1) * batchSize]])
    cursorMAG.execute(f'select * from abstracts where paperID in ({paperIDSetStr})')
    retMAG = cursorMAG.fetchall()
    abInfoList.extend(retMAG)
    cursorMAG.execute(f'select paperID, title from papers where paperID in ({paperIDSetStr})')
    retMAG = cursorMAG.fetchall()
    ttInfoList.extend(retMAG)
    # for item in retMAG:
    #     if len(item) == 1 or item[1] is None:
    #         abList.append('')
    #     else:
    #         abList.append(item[1].replace('\n', ' '))

if (i + 1) * batchSize < len(paperIDList):
    paperIDSetStr = ','.join([f'"{papID}"' for papID in paperIDList[(i + 1) * batchSize: ]])
    cursorMAG.execute(f'select * from abstracts where paperID in ({paperIDSetStr})')
    retMAG = cursorMAG.fetchall()
    abInfoList.extend(retMAG)
    cursorMAG.execute(f'select paperID, title from papers where paperID in ({paperIDSetStr})')
    retMAG = cursorMAG.fetchall()
    ttInfoList.extend(retMAG)
    
    # for item in retMAG:
    #     if len(item) == 1 or item[1] is None:
    #         abList.append(' ')
    #     else:
    #         abList.append(item[1].replace('\n', ' ').replace('\r', ' ').replace('  ', ' '))


In [5]:
import pandas as pd
dfAb = pd.DataFrame(abInfoList, columns=['paperID', 'abstract'])
df = pd.DataFrame(ttInfoList, columns=['paperID', 'title'])
df0 = pd.merge(df, dfAb, on='paperID', how='left')
df0.fillna('', inplace=True)
workDirMid = {'VCG': 'VCG', 'database': 'DB', 'visualization': 'VIS', 'CG': 'CG'}

In [7]:
len(df0[df0['abstract'] == ''])

68824

In [6]:
len(df0)

317802

In [6]:
workDir = f'./topicAll'

df0.to_csv(f'{workDir}/src/papers{workDirMid[field]}All.csv', index=False)


In [9]:
from datetime import datetime

nowDate = str(datetime.now().isoformat(sep=' ', timespec='milliseconds'))[2:10]
nowDate = nowDate.replace('-', '')
nowDate

'231111'

In [30]:
abInfoList
import json

with open(f'{workDir}/src/paperID2abstract.json', 'r') as f:
    paperID2Abstract = json.load(f)

df['abstract'] = df['paperID'].apply(lambda x: paperID2Abstract.get(x, ''))

In [7]:
df.fillna('', inplace=True)
len(df[df['abstract'] == ''])

809325

### fetch top papers in ./input/field/*

In [1]:
import pandas as pd
import os

srcDir = './topicTop/input'
resDir = './topicTop/src'
fieldList = ['visualization', 'CG', 'database']

for field in fieldList:
    df = None
    for fn in os.listdir(f'{srcDir}/{field}'):
        df0 = pd.read_csv(f'{srcDir}/{field}/{fn}')
        if df is None:
            df = df0
        else:
            df = pd.concat([df, df0])
    
    print(len(df))
    dfSingle = df.drop_duplicates(subset=['paperID'], keep='first')
    print(len(dfSingle))
    fieldAbb = {'visualization': 'VIS', 'CG': 'CG', 'database': 'DB'}
    dfSingle.to_csv(f'{resDir}/papers{fieldAbb[field]}Top.csv', index=False)

137832
80017
139771
77310
254938
157334


#### fetch title and abstract

In [29]:
import mysql.connector
conn = mysql.connector.connect(host="192.168.0.140",
                                user="root",
                                password="root",
                                database="MACG")
cursor = conn.cursor()

In [43]:
import os
import pandas as pd

paperIDInfoDt = {}
authorIDPaperIDDt = {}

resPre = './input/database'

for fn in  os.listdir(f'{resPre}/authorPaperID/'):
    if fn[-4:] !='.csv':
        continue
    
    
    authorID = fn[:-4]
    df = pd.read_csv(f'{resPre}/authorPaperID/{fn}')
    authorIDPaperIDDt[authorID] = list(df.paperID.values)

In [32]:
infoList = []
logPre = './log'

cnt = 0
for paperID in paperIDList:
    cursor.execute(f'select title from papers where paperID = "{paperID}"')
    ret = cursor.fetchone()
    if ret is None:
        with open(f'{logPre}/wrongTitle.txt', 'a+') as f:
            f.write(f'failed fetch title with {paperID}\n')
        title = ''
    else:
        title = ret[0]
    
    cursor.execute(f'select abstract from abstracts where paperID = "{paperID}"')
    ret = cursor.fetchone()
    if ret is None:
        with open(f'{logPre}/wrongAbstract.txt', 'a+') as f:
            f.write(f'failed fetch abstract with {paperID}\n')
        abstract = ''
    else:
        abstract = ret[0]
    
    paperIDInfoDt[paperID] = (title, abstract)
    infoList.append((paperID, title, abstract))
    
    if cnt % 10000 == 9999:
        print(f'{cnt} papers done')
    

In [44]:
dfPapers = pd.DataFrame(infoList, columns=['paperID', 'title', 'abstract'])
dfPapers.to_csv(f'{resPre}/papersInfoTop5K.csv', index=False)



In [45]:
print(f'{resPre}/papersInfoTop5K.csv')

./input/database/papersInfoTop5K.csv


In [1]:
import pandas as pd
resPre = './input/database'
srcPre = './src'
df = pd.read_csv(f'{srcPre}/papersInfoTop5K.csv')
df.fillna('', inplace=True)
df['documents'] = (df['title'] + '. ') * 3 +  df['abstract']
paperKey = df.paperID.tolist()
docs = df.documents.tolist()

print(len(paperKey))
print(len(docs))
print(len(df))

FileNotFoundError: [Errno 2] No such file or directory: './src/papersInfoTop5K.csv'

141202
141202
141202


In [ ]:
df = pd.read_csv(f'{directory}papersInfoTop5K.csv')
df.fillna('', inplace=True)

df['documents'] = (df['title'] + '. ') * 3 +  df['abstract']
paperKey = df.paperID.tolist()

docs = df.documents.tolist()

len(df)

In [12]:
for i, doc in enumerate(docs):
    if type(doc) != str:
        print(df.loc[df['paperID'] == paperKey[i]])
        break

#### judge functions in 

#### download embedding models

In [4]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id='paraphrase-MiniLM-L12-v2')

/home/xl/anaconda3/envs/py39base/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ConnectionError: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/models/paraphrase-MiniLM-L12-v2/revision/main (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fe413bd9dc0>: Failed to establish a new connection: [Errno 110] Connection timed out'))